In [ ]:
import sys
sys.path.insert(0, './Src/')


import os
from os import path, sep, mkdir
import numpy as np
import cv2
from cv2 import imread, imwrite
from matplotlib import pyplot as plt
import shutil
import re
import WSPL_func
from glob import glob
import json


# Initial information

In [ ]:
cwd = os.getcwd()
folder_dataset = f'{cwd}{sep}DataSet_MCF10AGFP'
if not path.exists(folder_dataset): print('Can not find the folder!')

## Genterate the train folder

In [ ]:
imglist = glob( f'{folder_dataset}{sep}*tif')
if len(imglist)>0:
    folder_path = path.dirname( imglist[0])
    WSPL_func.generate_TrainFolder(folder_path,imglist)

## Convert annotation format

In [ ]:
# 1: Nuclues; 2: background; 3: Edge
folder_train = f'{folder_dataset}{sep}Train_data'
WSPL_func.convert_Annoation( folder_train)

## Read the tranning data and generate the image features

In [ ]:
## parameter settings
cwd = os.getcwd()
folder_train = f'{folder_dataset}{sep}Train_data'
folder_model= f'{folder_train}{sep}model'
folder_images = f'{folder_train}{sep}images'
folder_eval = f'{folder_train}{sep}eval'
sigmaarray = np.array( [1,2,4,8]) #@WEKA
imgFS, y = WSPL_func.getTrainingData( folder_images, sigmaarray)
img_processing_method = np.array( WSPL_func.img_processing_method_template(sigmaarray) )
GT_opt = f'{folder_dataset}{sep}GT_Opt'
imgFS_GT, xy_total, imgGT = WSPL_func.getGTopt( GT_opt, sigmaarray)


## TWS CPU

In [ ]:
subFolderName = 'TWS_CPU'


modelInfo = {}
modelInfo['model_name'] = subFolderName
modelInfo['sigmaarray'] = sigmaarray.tolist()

sub_folder_model = f'{folder_model}{sep}{subFolderName}'
if not path.exists( sub_folder_model): mkdir( sub_folder_model)
# json_object = json.dumps(modelInfo, indent = 4)
with open(f"{sub_folder_model}{sep}Model_Info.json", "w+") as outfile:
    json.dump(modelInfo, outfile, indent=2)

cuml_model = WSPL_func.curfc(
    n_estimators=100,
)

cuml_model.fit(
        imgFS,
        y[:,2]-1
        )
WSPL_func.clear_output()


modelInfo_tmp = f'{sub_folder_model}{sep}Model_Info.json'
model_path = modelInfo_tmp.replace('.json', '.model')
WSPL_func.joblib.dump( cuml_model, model_path)

## Whole features (GTWS_whole)

In [ ]:
select = np.arange( imgFS.shape[1])
kf_num = 5
subFolderName = 'GTWS_whole'

test_n_setimators = np.array([50, 100, 150]).astype(int)
test_max_features = [2, 8, 14]
test_max_depth = np.array([8, 16, 24]).astype(int)
test_smaples_leaf = np.array([1, 3, 5]).astype(int)

parameters_array = WSPL_func.combin_paraterArray( test_n_setimators, test_max_features,\
    test_max_depth, test_smaples_leaf)

acc, opt_result = WSPL_func.opt_traing_parameters( select, parameters_array,test_max_features,\
        kf_num, imgFS, y, imgFS_GT, xy_total, imgGT)


test_n_setimators = np.array([100, 150, 200]).astype(int)
test_max_features = [2, 8, 14]
test_max_depth = np.array([4, 8, 12]).astype(int)
test_smaples_leaf = np.array([3, 5, 7]).astype(int)

parameters_array = WSPL_func.combin_paraterArray( test_n_setimators, test_max_features,\
    test_max_depth, test_smaples_leaf)

acc1, opt_result1 = WSPL_func.opt_traing_parameters( select, parameters_array,test_max_features,\
        kf_num, imgFS, y, imgFS_GT, xy_total, imgGT)

modelInfo = {}
modelInfo['model_name'] = subFolderName
modelInfo['sigmaarray'] = sigmaarray.tolist()

if acc > acc1:
    modelInfo['model_parameters_accuracy'] = acc
    modelInfo['model_parameters_name'] = opt_result[0]
    modelInfo['model_parameters_value'] = np.array( opt_result[1]).tolist()
else:
    modelInfo['model_parameters_accuracy'] = acc1
    modelInfo['model_parameters_name'] = opt_result1[0]
    modelInfo['model_parameters_value'] = np.array( opt_result1[1]).tolist()


sub_folder_model = f'{folder_model}{sep}{subFolderName}'
if not path.exists( sub_folder_model): mkdir( sub_folder_model)
# json_object = json.dumps(modelInfo, indent = 4)
with open(f"{sub_folder_model}{sep}Model_Info.json", "w+") as outfile:
    json.dump(modelInfo, outfile, indent=2)

## Forward selection (GTWS_selectedfeatures)

In [ ]:
# Find the selected features

subFolderName = 'GTWS_selectedfeatures'
sub_folder_model = f'{folder_model}{sep}{subFolderName}'
if not path.exists( sub_folder_model): mkdir( sub_folder_model)

kf_num = 5
Top = 9
select = WSPL_func.forwardSelection_opt(sub_folder_model, Top,\
    kf_num, imgFS, y, imgFS_GT, xy_total, imgGT)

modelInfo = {}
modelInfo['model_name'] = subFolderName
modelInfo['sigmaarray'] = sigmaarray.tolist()
modelInfo['select'] = select.tolist()
modelInfo['select_method'] = img_processing_method[select].tolist()

# json_object = json.dumps(modelInfo, indent = 4)
with open(f"{sub_folder_model}{sep}Model_Info.json", "w+") as outfile:
    json.dump(modelInfo, outfile, indent=2)

In [ ]:
# Opt parameters for RF

parameter_folder = f'{folder_model}{sep}{subFolderName}'
modelInfo = glob( f'{parameter_folder}{sep}*.json')[0]
with open(modelInfo, 'r') as openfile:
    modelInfo = json.load(openfile)
selec_all = np.array( modelInfo['select'], int)

for Top in np.arange( 3, selec_all.size+1):
    select = selec_all[:Top]
    test_n_setimators = np.array([50, 100, 150]).astype(int)
    test_max_features = ['auto']
    test_max_depth = np.array([8, 16, 24]).astype(int)
    test_smaples_leaf = np.array([1, 3, 5]).astype(int)

    parameters_array = WSPL_func.combin_paraterArray( test_n_setimators, test_max_features,\
    test_max_depth, test_smaples_leaf)

    acc, opt_result = WSPL_func.opt_traing_parameters( select, parameters_array,test_max_features,\
            kf_num, imgFS, y, imgFS_GT, xy_total, imgGT)


    test_n_setimators = np.array([100, 150, 200]).astype(int)
    test_max_features = ['auto']
    test_max_depth = np.array([4, 8, 12]).astype(int)
    test_smaples_leaf = np.array([3, 5, 7]).astype(int)

    parameters_array = WSPL_func.combin_paraterArray( test_n_setimators, test_max_features,\
    test_max_depth, test_smaples_leaf)

    acc1, opt_result1 = WSPL_func.opt_traing_parameters( select, parameters_array,test_max_features,\
            kf_num, imgFS, y, imgFS_GT, xy_total, imgGT)
    
    if acc > acc1:
        modelInfo['model_parameters_accuracy'] = acc
        modelInfo['model_parameters_name'] = opt_result[0]
        modelInfo['model_parameters_value'] = np.array( opt_result[1]).tolist()
    else:
        modelInfo['model_parameters_accuracy'] = acc1
        modelInfo['model_parameters_name'] = opt_result1[0]
        modelInfo['model_parameters_value'] = np.array( opt_result1[1]).tolist()

    modelInfo['select'] = select.tolist()
    # json_object = json.dumps(modelInfo, indent = 4)
    with open(f"{parameter_folder}{sep}Model_Info_{Top:02d}.json", "w+") as outfile:
        json.dump(modelInfo, outfile, indent=2)

## Train the model

In [ ]:
subFolderName = 'GTWS_whole'
sub_folder_model = f'{folder_model}{sep}{subFolderName}'
modelInfo_tmp = f'{sub_folder_model}{sep}Model_Info.json'
WSPL_func.train_model(imgFS , y, modelInfo_tmp)

In [ ]:
subFolderName = 'GTWS_selectedfeatures'
sub_folder_model = f'{folder_model}{sep}{subFolderName}'
for itop in range( 3,Top+1):
    modelInfo_tmp = f'{sub_folder_model}{sep}Model_Info_{itop:02d}.json'
    WSPL_func.train_model(imgFS , y, modelInfo_tmp)
